# Hito 2

## Aplicación modelo \textit{Black and Scholes}

Antes de comenzar con la valorización de la opción cuyo activo subyacente es la acción del Banco de Chile, se presentarán cuáles son los parámetros que este modelo necesita, y la fórmula respectiva. Los parámetros para el modelo son:
\begin{itemize}
\item $S$: Precio actual de la acción.
\item $K$: Precio del ejercicio o \textit{strike price} de la opción.
\item $t$: Tiempo que falta para el vencimiento (en años).
\item $\sigma$: Variación del rendimiento de la acción.
\item $R$: La tasa libre de riesgo.
\end{itemize}
Luego, la fórmula para el modelo continuo es la siguiente:
\begin{equation}
\label{eq:BS}
C(S,t) = S\cdot \phi(d_1) - K\cdot \exp(-R\cdot T) \cdot \phi(d_{2})\;,
\end{equation}
donde,
\begin{align*}
d_1 &= \frac{\log\left(\frac{S}{K}\right) + \left(R + \frac{\sigma^2}{2}\right)\cdot t}{\sqrt{\sigma^2 \cdot t}}\;,\\
d_2 &= d_1 - \sqrt{\sigma^2 \cdot t}\;.
\end{align*}
Entonces, teniendo los parámetros y la fórmula a utilizar, se puede realizar el siguiente código.

In [ ]:
import numpy as np
import scipy.stats as si
import yfinance as yf

def volatilidad(ticker, periodo_t):
    df = yf.download(ticker, period = '1y')

    retornos_diarios = df['Adj Close'].pct_change()

    sigma = retornos_diarios.rolling(periodo_t).std()*np.sqrt(periodo_t)
    # Volatilidad anualizada
    return sigma

def black_scholes_call(S, K, T, r, sigma):
    # S: Precio actual del activo subyacente.
    # K: Precio del ejercicio de la opción.
    # T: Tiempo de que falta para la fecha de vencimiento.
    # r: Tasa libre de riesgo.
    # sigma: Volatilidad del activo subyacente.
    
    d1 = (np.log(S / K) + (r + 0.5 * sigma ** 2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    call = (S * si.norm.cdf(d1, 0.0, 1.0) - K * np.exp(-r * T) * si.norm.cdf(d2, 0.0, 1.0))
    return call

ticker = 'CHILE.SN'
data = yf.download(ticker, period='1d')

# Precio actual de la acción del Banco de Chile.
S = data['Close'][-1]

# Precio del ejercicio.
K = 100

# Tiempo que falta para la fecha de vencimiento (en años)
T = 1

# Tasa libre de riesgo.
r = 0.02

# Volatilidad
sigma = 0.2

# Calculate call option price
call_option_price = black_scholes_call(S, K, T, r, sigma)

print(f'El precio para una opción call es: {call_option_price}')